In [ ]:
!pip install roboflow
!pip install ultralytics==8.3.40
!pip install streamlit ngrok opencv-python-headless
!pip install pyngrok
# !ngrok config add-authtoken xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx

import os

from pyngrok import ngrok
from google.colab import drive
drive.mount('/content/gdrive')

# Simulation of the smart fridge

In [ ]:
%%writefile smart_fridge_app.py
import cv2
import os
import glob
import random
import shutil
import json

import matplotlib.pyplot as plt
import streamlit as st
import numpy as np

from google.colab.patches import cv2_imshow
from ultralytics import YOLO

def setup_yolo_level1():
    model1 = YOLO("/content/gdrive/My Drive/Georgios_Karagiannis/Code/Level1_train/detect/train/weights/best.pt")
    return model1

def setup_yolo_level2():
    model2 = YOLO("/content/gdrive/My Drive/Georgios_Karagiannis/Code/Level2_train/detect/train/weights/best.pt")
    return model2

def analyze_image(image, temperature, humidity, yolo_model_level1, yolo_model_level2):
    yolo_results_level1 = yolo_model_level1(image)
    fruit_classes = [yolo_model_level1.names[int(cls)] for cls in yolo_results_level1[0].boxes.cls]

    if "Tomato" in fruit_classes:
        yolo_results_level2 = yolo_model_level2(image)
        ripeness_classes = [yolo_model_level2.names[int(cls)] for cls in yolo_results_level2[0].boxes.cls]
        ripeness_message = []

        for ripeness in ripeness_classes:
            if ripeness == "over-ripe":
                ripeness_message.append("Over-ripe tomatoes detected. Discard them immediatley before they start to effect the other fruits.")
            elif ripeness == "ripe":
                if 7 <= temperature <= 10 and humidity > 70:
                    ripeness_message.append("Ripe tomatoes detected. They are edible for up to 4 days. If you want your tomatoes to last longer, you should store them somewhere cooler.")
                elif temperature < 7:
                    ripeness_message.append("You should put your ripe tomatoes somewhere warmer to avoid chilling injuries.")
                else:
                    ripeness_message.append("Your tomatoes will ripen quickly, but you should bear in mind that they are susceptible to pathogens.")
            elif ripeness == "half-ripe":
                if 8 <= temperature <= 10 and humidity > 60:
                    ripeness_message.append("Half-ripe tomatoes detected. They are not edible but can be stored for up to 7 days untill they are edible.If you want to ripe them more slower , you should store them somewhere cooler and with less humidity")
                elif temperature < 8:
                    ripeness_message.append("Your rippening prosses will be stoped and your tomatoes will be susceptible to chilling injuries.")
                else:
                    ripeness_message.append("Your tomatoes will ripen quickly, but you should bear in mind that they are susceptible to pathogens.")
            elif ripeness == "unripe":
                if 8 <= temperature <= 12 and humidity > 50:
                    ripeness_message.append("Unripe tomatoes detected. They are not edible but can be stored for up to 10 days untill they are edible.If you want to ripe them more slower , you should store them somewhere cooler and with less humidity")
                elif temperature < 8:
                    ripeness_message.append("Your rippening prosses will be stoped and your tomatoes will be susceptible to chilling injuries.")
                else:
                    ripeness_message.append("Your tomatoes will ripen quickly, but you should bear in mind that they are susceptible to pathogens.")
        return ripeness_message
    else:
        remaining_items = [item for item in fruit_classes if item != "Tomato"]
        return f"No tomatoes detected. Items in the fridge: {', '.join(remaining_items)}"

def main():
    st.markdown(
        "<h1 style='text-align: center;'>Welcome back</h1>",
        unsafe_allow_html=True,
    )
    temperature = st.number_input(
        "Temperature (°C)", value=8.0, min_value=-10.0, max_value=15.0, step=1.0
    )
    humidity = st.number_input(
        "Humidity (%)", value=75.0, min_value=0.0, max_value=100.0, step=5.0
    )

    st.markdown(
        "<h3 style='text-align: center;'>What would you like to do?</h3>",
        unsafe_allow_html=True,
    )

    if "selected_image" not in st.session_state:
        simulation_image_path = "/content/gdrive/MyDrive/Georgios_Karagiannis/Code/simulation"
        image_files = glob.glob(os.path.join(simulation_image_path, "*.[jp][pn]g"))
        if not image_files:
            st.error("No images found in the test directory.")
            st.stop()
        st.session_state.selected_image = random.choice(image_files)

    selected_image = st.session_state.selected_image

    # selected_image = "/content/gdrive/MyDrive/Georgios_Karagiannis/Code/simulation/sim6.jpg"

    if st.button("See what's inside"):
        yolo_model_level1 = setup_yolo_level1()

        img = cv2.imread(selected_image)

        results = yolo_model_level1([img])

        for result in results:
            result.save(filename="output.jpg")

        st.success("Analysis Complete!")
        st.image("output.jpg", caption="Detected Items in the Fridge", use_container_width=True)
        fruit_classes = [yolo_model_level1.names[int(cls)] for cls in results[0].boxes.cls]
        fruits_counts = {fruits: fruit_classes.count(fruits) for fruits in set(fruit_classes)}

        st.write("Contents of the fridge:")
        for fruits, count in fruits_counts.items():
            st.write(f"- {count} {fruits}(s)")

    if st.button("See the maturity"):

        yolo_model_level1 = setup_yolo_level1()
        yolo_model_level2 = setup_yolo_level2()

        img = cv2.imread(selected_image)

        results = yolo_model_level2([img])

        for result in results:
            result.save(filename="output.jpg")

        ripeness_analysis = analyze_image(selected_image, temperature, humidity, yolo_model_level1, yolo_model_level2)

        st.success("Analysis Complete!")
        st.image("output.jpg", caption="Ripeness of tomatoes detected", use_container_width=True)

        if isinstance(ripeness_analysis, list):
            st.write("Here’s how you can handle the maturity levels:")
            for msg in ripeness_analysis:
                st.write("- " + msg)
        else:
            st.write(ripeness_analysis)

if __name__ == "__main__":
    main()

In [ ]:
os.system("streamlit run smart_fridge_app.py &")

public_url = ngrok.connect(8501)
print(f"Public URL: {public_url}")